## 필요한 것들
1. clf 폴더 안에 모델파일
2. preprocessed 폴더 안에 classification 안에 tset_1 폴더 안에에 0,1,2,3,4 폴더 각각 하나씩
3. 사진 링크, 사진 저장할 "0" 폴더 링크
4. 1,2,3,4에 사진 파일 각각 하나씩 <- 근데 어차피 쓰이지 않음.

## 과정
1. 소도체 단면 사진을 링크에서 받아와서 폴더에 저장
2. 모델과 사진 폴더를 불러옴
3. TEST
4. 링크에서 온 사진 "0" 폴더에 저장되지만 어차피 우리는 예측된 등급 결과와 사진만 출력할거라서 상관 없음
5. 예측된 등급과 사진 출력

In [ ]:
pip install coremltools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.6 MB/s eta 0:00:00


## 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 패키지 불러오기

In [2]:
pip install torchinfo

In [3]:
import os
import os.path as osp
import sys
from glob import glob

import datetime
import time
import pytz

import pandas as pd
import numpy as np
from tqdm import tqdm

import sklearn

import torch
import torchvision
from torchinfo import summary

import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision import datasets, transforms
import torchvision.models as models

# import explain

## 이미지, 모델 불러오기

In [4]:
import requests
import os

def save_image_from_url(image_url, save_path):
    try:
        # 이미지 URL에서 이미지 다운로드
        response = requests.get(image_url)
        response.raise_for_status()

        # 저장할 폴더가 존재하지 않는 경우 생성
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        # 이미지를 바이너리 모드로 저장
        with open(save_path, 'wb') as f:
            f.write(response.content)

        print(f"이미지가 다음 경로에 저장되었습니다: {save_path}")

    except requests.exceptions.HTTPError as errh:
        print(f"HTTP 오류 발생: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"연결 오류 발생: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"타임아웃 오류 발생: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"기타 오류 발생: {err}")

image_url = "https://www.amnews.co.kr/news/photo/201807/33411_22979_495.jpg"
save_path = "/content/drive/MyDrive/fucking_cow/project_files/New_sample/Dataset/preprocessed/classification/test_1/0/want_to_pred.jpg"

save_image_from_url(image_url, save_path)

이미지가 다음 경로에 저장되었습니다: /content/drive/MyDrive/fucking_cow/project_files/New_sample/Dataset/preprocessed/classification/test_1/0/want_to_pred.jpg


In [5]:
# 모델에 입력할 Input 이미지의 크기를 지정함.
IMG_WIDTH = 512
IMG_HEIGHT = 512

# 배치 사이즈 임의 지정
batch_size = 4

# 데이터를 읽어올 경로를 지정함.
data_dir='/content/drive/MyDrive/fucking_cow/project_files/New_sample/Dataset/preprocessed/classification/'
# 학습된 모델이 저장된 경로를 지정함.
ckpt_dir = '/content/drive/MyDrive/fucking_cow/project/QC_cow1/classification/model_weight/'#'/content/drive/MyDrive/fucking_cow/project/QC_cow1/classification/checkpoint/'

# 모델 이름
Model_name = 'resnext50_32x4d_cow_classification_v1_best_Loss'#'resnext50_32x4d_cow_classification_v1'
# 모델을 연산할 장비 설정 (Docker환경에서 작동할 수 있도록 CPU에서 계산함)
device = torch.device('cpu')

In [6]:
# 이미지 transformation
test_compose=transforms.Compose([
    transforms.Resize((IMG_HEIGHT,IMG_WIDTH)),
    transforms.ToTensor()
])

# Test set을 읽어옴
# test_dataset=datasets.ImageFolder(os.path.join(data_dir, 'test_1'), transform=test_compose)
# test_dataloader=DataLoader(test_dataset, batch_size=batch_size, pin_memory=True, shuffle=False)

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
from torchvision.transforms import Compose, Resize, ToTensor, Normalize

from PIL import Image

def pil_loader(path):
    # 이미지 로드
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

test_dataset = ImageFolder(os.path.join(data_dir, 'test_1'), loader=pil_loader, transform=test_compose)
# test_dataset = ImageFolder(os.path.join(data_dir, 'test_1'), loader=default_loader, transform=test_compose)
include_list = [0, 5, 6, 7, 8, 9]
included_indices = [i for i, (_, label) in enumerate(test_dataset.samples) if label in include_list]
test_subset = Subset(test_dataset, included_indices)

test_dataloader = DataLoader(test_subset, batch_size=batch_size, pin_memory=True, shuffle=False)

In [7]:
# Model: ResNeXt50_32x4d
model=models.resnext50_32x4d(pretrained=False)
# 최종 Label 의 개수 = 5 (1++, 1+, 1, 2, 3)
model.fc=nn.Linear(model.fc.in_features, 5)

# 학습된 모델 가중치 불러오기
checkpoint=torch.load(ckpt_dir + Model_name + '.pth', map_location=device)
model.load_state_dict(checkpoint['state_dict'])

# print(model.state_dict())
# for key in checkpoint['state_dict']:
#     print(key)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [9]:
_ = summary(model, (4, 3, 512, 512))

_

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [4, 5]                    --
├─Conv2d: 1-1                            [4, 64, 256, 256]         9,408
├─BatchNorm2d: 1-2                       [4, 64, 256, 256]         128
├─ReLU: 1-3                              [4, 64, 256, 256]         --
├─MaxPool2d: 1-4                         [4, 64, 128, 128]         --
├─Sequential: 1-5                        [4, 256, 128, 128]        --
│    └─Bottleneck: 2-1                   [4, 256, 128, 128]        --
│    │    └─Conv2d: 3-1                  [4, 128, 128, 128]        8,192
│    │    └─BatchNorm2d: 3-2             [4, 128, 128, 128]        256
│    │    └─ReLU: 3-3                    [4, 128, 128, 128]        --
│    │    └─Conv2d: 3-4                  [4, 128, 128, 128]        4,608
│    │    └─BatchNorm2d: 3-5             [4, 128, 128, 128]        256
│    │    └─ReLU: 3-6                    [4, 128, 128, 128]        --
│  

## 테스트 수행

In [10]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 시작 시간 : {nowDatetime}')

테스트 시작 시간 : 2023-10-13 18:39:27


In [11]:
since=time.time()

In [12]:
tot_labels=[]  # 전체 정답 Label 을 저장할 리스트
tot_pred_labels=[]  # 전체 예측 Label 을 저장할 리스트

# 모델을 device에 올림
model=model.to(device)
# 네트워크를 evaluation 용으로 선언
model=model.eval()

# test 이기 때문에 backprop 진행 x
with torch.no_grad():
    for images, labels in tqdm(test_dataloader):

        images=images.to(device)
        labels=labels.to(device)

        # Model Prediction (forward)
        output=model(images)
        _, output_index = torch.max(output, 1)

        tot_labels.extend(list(labels.numpy()))
        tot_pred_labels.extend(list(output_index.view(-1).numpy()))

100%|██████████| 1/1 [00:01<00:00,  1.75s/it]


In [13]:
end=time.time()
print(f'테스트 소요 시간: {int((end-since)//60)}m {int((end-since)%60)}s')

테스트 소요 시간: 0m 7s


In [14]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 종료 시간 : {nowDatetime}')

테스트 종료 시간 : 2023-10-13 18:39:37


## 결과 출력하기

In [15]:
import os

# test_dataset = test_dataloader.dataset
# test_filenames = [os.path.basename(x[0]) for x in test_dataset.imgs]
# test_labels = [x[1] for x in test_dataset.imgs]

correct_filenames = []
not_correct_filenames = []
for i in range(len(tot_labels)):
    if tot_labels[i] == tot_pred_labels[i]:
        filename = os.path.basename(test_subset.dataset.imgs[included_indices[i]][0])
        correct_filenames.append(filename)
    else:
        filename = os.path.basename(test_subset.dataset.imgs[included_indices[i]][0])
        not_correct_filenames.append(filename)


print(correct_filenames)
print(not_correct_filenames)
# correct_filenames에는 정답 라벨과 예측 라벨이 같은 이미지들의 파일명이 저장됩니다.
# not_correct_filenames에는 다른.

[]
['want_to_pred.jpg']


In [16]:
real_labels = ['1++', '1+', '1', '2', '3']

print("실제등급 : "+real_labels[tot_labels[0]])
print("예측등급 : "+real_labels[tot_pred_labels[0]])

print("AI로 예측한 소도체의 등급은 "+real_labels[tot_pred_labels[0]]+"등급입니다.")

실제등급 : 1++
예측등급 : 1
AI로 예측한 소도체의 등급은 1등급입니다.


In [22]:
from PIL import Image

# for filename in correct_filenames:
#     img = Image.open(os.path.join(data_dir, 'test_1', '*', filename))
#     img.show()


import glob

# for filename in correct_filenames:
#     img_paths = glob.glob(os.path.join(data_dir, 'test_1', '*', filename))
#     splited = filename.split("_")
#     print("소도체의 등급은 "+splited[3]+"등급입니다.")
#     for img_path in img_paths:
#         img = Image.open(img_path)
#         img.show()

for filename in not_correct_filenames:
    img_paths = glob.glob(os.path.join(data_dir, 'test_1', '*', filename))
    print("소도체의 등급은 "+real_labels[tot_pred_labels[0]]+"등급입니다.")
    for img_path in img_paths:
        img = Image.open(img_path)
        img.show()

# 실제 등급과 예측등급이 같으면 correct_filenames 코드를, 다르면 not_correct_filenames 코드 이용하면 됨.

소도체의 등급은 1등급입니다.


**아래는 코드 실험입니다!!**

In [ ]:
import torch
import numpy as np
from PIL import Image

# # Model: ResNeXt50_32x4d
# model=models.resnext50_32x4d(pretrained=False)
# # 최종 Label 의 개수 = 5 (1++, 1+, 1, 2, 3)
# model.fc=nn.Linear(model.fc.in_features, 5)

# # 학습된 모델 가중치 불러오기
# checkpoint=torch.load(ckpt_dir + Model_name + '.pth', map_location=device)
# model.load_state_dict(checkpoint['state_dict'])

# 1. 모델 로드
model_path = '/content/drive/MyDrive/fucking_cow/project/QC_cow1/classification/model_weight/saved_model.pth'
model = torch.load(model_path)
model.eval()

# 2. 입력 이미지 로드 및 전처리
img_path = '/content/drive/MyDrive/fucking_cow/project_files/New_sample/Dataset/preprocessed/classification/test_1/2/QC_cow_segmentation_1_074512_1.jpg'
img = Image.open(img_path)
img = img.resize((512, 512))  # 이미지 크기 조정
img_array = np.array(img)     # numpy 배열로 변환
img_array = np.transpose(img_array, (2, 0, 1)) # 차원 순서 변경
img_tensor = torch.from_numpy(img_array).float().unsqueeze(0)  # 입력 텐서로 변환
# img_tensor = img_tensor.repeat(4, 1, 1, 1)  # 4개의 이미지를 입력으로 사용하기 위해 repeat 함수를 이용하여 복사

# 3. 모델에 입력 이미지 전달하여 결과 예측
tot_pred_labels = []

with torch.no_grad():
    output = model(img_tensor)

# 4. 결과 출력
print(output)

_, output_index = torch.max(output, 1)
tot_pred_labels.extend(list(output_index.view(-1).numpy()))

print(tot_pred_labels)
# train_data = torch.from_numpy(np_train_data).float()